## LSTM-Inception Text Classification with Keras

In [1]:
%run Setup.ipynb
%run ExtraFunctions.ipynb

[i] Loaded Parameters:
 40000 35 0.2 200 
 dataset/glove/glove.twitter.27B.200d.txt
[i] Importing Modules...


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[+] Using Keras version 2.1.4
[i] Finished Importing Modules
[i] . Reading from csv file...Done!
[i] Found 30447 unique tokens.
[+] Shape of data tensor: (40000, 30)
[+] Shape of label tensor: (40000, 4)
[+] Number of entries in each category:
[+] Training:
 [ 7708. 10867. 12305.  1120.]
[+] Validation:
 [1936. 2757. 2994.  313.]
[i] Loading GloVe from: dataset/glove/glove.twitter.27B.200d.txt ...Done.
[+] Proceeding with Embedding Matrix...Completed!
Finished running setup.


Load embedding matrix into an `Embedding` layer. Toggle `trainable=False` to prevent the weights from being updated during training.

In [2]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

### LSTM + Inception Structure
[Reference](https://github.com/richliao/textClassifier), [LTSM](http://colah.github.io/posts/2015-08-Understanding-LSTMs/), [CNN Source](https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html) and [Notes](http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/)

Deeper CNN as described in [CNN for Sentence Classification](http://www.aclweb.org/anthology/D14-1181) (Yoon Kim, 2014), multiple filters have been applied. This can be implemented using Keras `Merge` Layer.

In [3]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

l_lstm1 = Bidirectional(LSTM(4,dropout=0.3,recurrent_dropout=0.3,return_sequences=True))(embedded_sequences)

# inception module
inception, filter_sizes = [], [3,4,6,7]
for fsz in filter_sizes:
    l_conv_i1 = Conv1D(filters=12,kernel_size=1,
                    activation='relu',)(l_lstm1)
    l_conv_i2 = Conv1D(filters=12,kernel_size=fsz,
                       activation='relu',kernel_regularizer=regularizers.l2(0.01))(l_conv_i1)
    inception.append(l_conv_i2)
l_pool_inc = MaxPooling1D(4)(l_lstm1)
l_conv_inc = Conv1D(filters=12,kernel_size=1,
                    activation='relu',kernel_regularizer=regularizers.l2(0.02))(l_pool_inc)
inception.append(l_conv_inc)

l_merge = Concatenate(axis=1)(inception)
l_pool1 = MaxPooling1D(3)(l_merge)
l_drop1 = Dropout(0.4)(l_pool1)
l_flat = Flatten()(l_drop1)
l_dense = Dense(16, activation='relu',kernel_regularizer=regularizers.l2(0.02))(l_flat)

preds = Dense(4, activation='softmax')(l_dense)

In [4]:
model = Model(sequence_input, preds)
adadelta = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
lr_metric = get_lr_metric(adadelta)
#keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
model.compile(loss='categorical_crossentropy',
              optimizer=adadelta,
              metrics=['acc', lr_metric])

In [5]:
tensorboard = callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=16, write_grads=True , write_graph=True)
model_checkpoints = callbacks.ModelCheckpoint("checkpoint.h5", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
lr_schedule = callbacks.LearningRateScheduler(initial_boost)

In [6]:
model.summary()
model.save('ltsm-inception.h5')
print("Training Progress:")
model_log = model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=200, batch_size=50,
          callbacks=[tensorboard, lr_schedule])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 30, 200)      6089600     input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 30, 8)        6560        embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 30, 12)       108         bidirectional_1[0][0]            
__________________________________________________________________________________________________
conv1d_3 (

Epoch 22/200
32000/32000 [==============================] - 85s 3ms/step - loss: 0.9264 - acc: 0.6315 - lr: 1.0000 - val_loss: 1.0215 - val_acc: 0.5764 - val_lr: 1.0000
Epoch 23/200
32000/32000 [==============================] - 84s 3ms/step - loss: 0.9230 - acc: 0.6343 - lr: 1.0000 - val_loss: 1.0210 - val_acc: 0.5756 - val_lr: 1.0000
Epoch 24/200
10250/32000 [========>.....................] - ETA: 57s - loss: 0.9092 - acc: 0.6455 - lr: 1.0000

KeyboardInterrupt: 

In [ ]:
model.save('ltsm-inception.h5')

In [ ]:
import pandas as pd
pd.DataFrame(model_log.history).to_csv("history-inception.csv")